## Задание 2
Имеются результаты A/B теста, в котором двум группам пользователей предлагались различные наборы акционных предложений. Известно, что ARPU в тестовой группе выше на 5%, чем в контрольной. При этом в контрольной группе 1928 игроков из 202103 оказались платящими, а в тестовой – 1805 из 202667.

Какой набор предложений можно считать лучшим? Какие метрики стоит проанализировать для принятия правильного решения и как?

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

file_id = '1FIxdHQYWYmpypdiwfgUMBL06IOZJepyx'
download_url = f'https://drive.google.com/uc?id={file_id}&export=download'

df = pd.read_csv(download_url, sep=';')
assert {'user_id', 'revenue', 'testgroup'}.issubset(df.columns)

agg = (
    df.groupby('testgroup')
      .agg(users       = ('user_id',   'nunique'),
           payers      = ('revenue',   lambda s: (s > 0).sum()),
           revenue_sum = ('revenue',   'sum'))
)

agg['PR']    = agg['payers']  / agg['users']
agg['ARPU']  = agg['revenue_sum'] / agg['users']
agg['ARPPU'] = agg['revenue_sum'] / agg['payers'].clip(lower=1)

display(agg)

ctrl = df[df['testgroup'] == 'a']
test = df[df['testgroup'] == 'b']

# ARPU: bootstrap 95% CI
def bootstrap_diff(x, y, iters=10_000):
    diffs = []
    n1, n2 = len(x), len(y)
    for _ in range(iters):
        diffs.append(np.mean(np.random.choice(x, n1, True))
                     - np.mean(np.random.choice(y, n2, True)))
    return np.percentile(diffs, [2.5, 97.5]), np.mean(diffs)

ci_arpu, diff_arpu = bootstrap_diff(test['revenue'], ctrl['revenue'])
print(f'ΔARPU = {diff_arpu:+.4f}  (95% CI {ci_arpu[0]:.4f} … {ci_arpu[1]:.4f})')

# payer-rate: Z-test
succ_a, n_a = agg.loc['a', ['payers', 'users']]
succ_b, n_b = agg.loc['b', ['payers', 'users']]
p_pool = (succ_a + succ_b) / (n_a + n_b)
se     = np.sqrt(p_pool * (1 - p_pool) * (1/n_a + 1/n_b))
z      = (succ_b/n_b - succ_a/n_a) / se
p_pr   = 2 * (1 - stats.norm.cdf(abs(z)))
print(f'ΔPR  = {(succ_b/n_b - succ_a/n_a):+.4%},  Z = {z:.2f},  p = {p_pr:.4f}')

# ARPPU: Mann–Whitney U
arppu_a = ctrl.loc[ctrl['revenue']>0, 'revenue']
arppu_b = test.loc[test['revenue']>0, 'revenue']
u_stat, p_arppu = stats.mannwhitneyu(arppu_a, arppu_b, alternative='two-sided')
print(f'MWU for ARPPU: U = {u_stat:.0f},  p = {p_arppu:.4f}')

print(f"ARPU uplift: {diff_arpu/agg.loc['a','ARPU']:+.2%}")
print(f"P-value ARPU bootstrap: {'significant' if ci_arpu[0]*ci_arpu[1] > 0 else 'NOT significant'}")
print(f"P-value PR Z-test:      {'significant' if p_pr < .05 else 'NOT significant'}")
print(f"P-value ARPPU MWU:      {'significant' if p_arppu < .05 else 'NOT significant'}")

,users,payers,revenue_sum,PR,ARPU,ARPPU
testgroup,,,,,,
a,202103,1928,5136189,0.009540,25.413720,2663.998444
b,202667,1805,5421603,0.008906,26.751287,3003.658172


ΔARPU = +1.3141  (95% CI -2.9701 … 5.3440)
ΔPR  = -0.0633%,  Z = -2.11,  p = 0.0350
MWU for ARPPU: U = 222015,  p = 0.0000
ARPU uplift: +5.17%
P-value ARPU bootstrap: NOT significant
P-value PR Z-test:      significant
P-value ARPPU MWU:      significant


## Какой набор предложений можно считать лучшим?

На основании проведенного A/B-теста и анализа ключевых метрик, **ни один из наборов предложений нельзя однозначно признать лучшим на данный момент.**

Анализ показывает следующее:
*   **Набор B (тестовый)** продемонстрировал рост `ARPU` (среднего дохода на пользователя) на **+5.17%** (ΔARPU = +1.3141) по сравнению с контрольным набором A. Однако это изменение **не является статистически значимым**. Нельзя с достаточной уверенностью утверждать, что предложения B действительно увеличивают общий доход.
*   Одновременно с этим, **набор B** привел к **статистически значимому снижению `PR` (Payer Rate / доли платящих пользователей)** на 0.0633 процентных пункта (p = 0.0350). Это негативный результат, так как меньше пользователей совершают покупку.
*   Несмотря на то, что у **набора B** **статистически значимо выше `ARPPU` (средний доход на платящего пользователя)** (p = 0.0000), что указывает на то, что платящие пользователи тратят больше, это увеличение среднего чека не смогло компенсировать падение конверсии до такой степени, чтобы обеспечить статистически значимый рост общего `ARPU`.

**Вывод:** Поскольку основная целевая метрика (`ARPU`) не показала статистически значимого улучшения, а важная guard-rail метрика (`PR`) статистически значимо ухудшилась, **набор предложений B не может быть признан лучшим.** Рекомендуется не внедрять изменения группы B и оставить текущий набор предложений A.


## Какие метрики стоит проанализировать для принятия правильного решения и как?

Для принятия обоснованного решения об эффективности акционных предложений необходимо проанализировать следующие ключевые метрики:

1.  **`ARPU` (Average Revenue Per User) — Целевая метрика:**
    *   Нужно сравнить средние значения `ARPU` между тестовой и контрольной группами. Провести тест на статистическую значимость разницы (bootstrap-тест для средних, или t-тест после логарифмирования для нормализации распределения выручки).
    *   **Цель:** Обнаружить статистически значимое увеличение `ARPU` в тестовой группе.

2.  **`PR` (Payer Rate / Conversion Rate) — Ключевая guard-rail метрика:**
    *   Сравнить долю пользователей, совершивших покупку, между группами. Использовать тест на статистическую значимость для пропорций (Z-тест для двух пропорций или тест хи-квадрат).
    *   **Цель:** Убедиться, что `PR` в тестовой группе не показывает статистически значимого снижения. `PR` должен оставаться на прежнем уровне или расти.

3.  **`ARPPU` (Average Revenue Per Paying User) — Диагностическая метрика:**
    *   Сравнить средний доход, полученный только с платящих пользователей, между группами. Ввиду часто ненормального распределения сумм платежей, следует использовать непараметрические тесты (тест Манна-Уитни) или bootstrap для сравнения средних/медиан.
    *   **Цель:** Понять, за счет чего происходят изменения в `ARPU`. Рост `ARPPU` положителен, но должен рассматриваться в комплексе с `PR`.

**Общий подход к принятию решения:**
Решение о внедрении нового набора предложений принимается, если наблюдается **статистически значимое увеличение `ARPU`** при условии, что **`PR` не показывает статистически значимого падения**, а другие важные метрики (удержание клиентов, уровень возвратов) также остаются на приемлемом уровне или улучшаются. Если `ARPU` не растет значимо или `PR` падает, от внедрения стоит отказаться.

## Задание 3

В игре Plants & Gardens каждый месяц проводятся тематические события, ограниченные по времени. В них игроки могут получить уникальные предметы для сада и персонажей, дополнительные монеты или бонусы. Для получения награды требуется пройти ряд уровней за определенное время. С помощью каких метрик можно оценить результаты последнего прошедшего события?

Предположим, в другом событии мы усложнили механику событий так, что при каждой неудачной попытке выполнения уровня игрок будет откатываться на несколько уровней назад. Изменится ли набор метрик оценки результата? Если да, то как?

### С помощью каких метрик можно оценить результаты последнего прошедшего события?

Цели тематического события обычно включают:
1.  **Вовлечение игроков (Engagement):** Заставить игроков активно провести время в игре.
2.  **Удержание игроков (Retention):** Мотивировать игроков вернуться в игру и продолжить играть после события.
3.  **Монетизация (Monetization):** Прямая (покупка пропусков, бустеров для события) или косвенная (повышение лояльности, ведущее к будущим покупкам).
4.  **Предоставление контента:** Дать игрокам новый опыт и награды.

Исходя из этих целей, можно выделить следующие группы метрик:

**1. Метрики участия и вовлеченности (Participation & Engagement):**
    *   **Event Participation Rate (Коэффициент участия в событии):** `(Количество игроков, начавших событие / Общее количество активных игроков за период события) * 100%`. Показывает, насколько событие было интересно аудитории.
    *   **Event Completion Rate (Коэффициент завершения события):** `(Количество игроков, полностью прошедших событие / Количество игроков, начавших событие) * 100%`. Отражает, насколько достижимой была цель события и насколько игроки были мотивированы дойти до конца.
    *   **Average Levels Completed per Participant (Среднее количество пройденных уровней на одного участника):** Показывает глубину вовлечения тех, кто участвовал.
    *   **Funnel Analysis (Анализ воронки прохождения уровней):** Распределение игроков по максимальному достигнутому уровню в событии. Помогает выявить "узкие места" или слишком сложные этапы.
    *   **Time Spent in Event (Время, проведенное в событии):** Среднее время, которое игроки посвятили уровням события.
    *   **Session Length / Frequency during Event (Длина / Частота сессий во время события):** Изменились ли эти показатели по сравнению с обычным периодом для участников события?

**2. Метрики влияния на основные игровые показатели:**
    *   **DAU/MAU (Daily/Monthly Active Users - Дневная/Месячная активная аудитория):** Изменилась ли общая активность в игре во время проведения события.
    *   **Retention Rate (Коэффициент удержания):** Сравнить удержание игроков, участвовавших в событии, с теми, кто не участвовал, а также с показателями до события.
    *   **Churn Rate (Уровень оттока):** Не вызвал ли ивент негативной реакции, приведшей к уходу игроков (особенно если он был слишком сложным или неудачным).

**3. Метрики монетизации (если применимо):**
    *   **Revenue from Event-Specific Offers (Доход от специальных предложений, связанных с событием):** Если были платные бустеры, пропуски, энергия для события.
    *   **Conversion to Paying User (Конверсия в платящего пользователя):** Если событие привлекло новых игроков или мотивировало неплатящих совершить первую покупку.

**4. Метрики удовлетворенности (Qualitative Metrics):**
    *   **Player Feedback (Обратная связь от игроков):** Отзывы в сообществах, опросы в игре, оценки в сторах.
    *   **Social Media Sentiment (Настроения в социальных сетях):** Обсуждение события, его положительные и отрицательные стороны.

---

###  Изменится ли набор метрик оценки результата? Если да, то как?

**Да, набор метрик изменится и дополнится.** Основные метрики останутся важными, но акценты сместятся, и появятся новые, специфичные для данной механики.

**Изменения и дополнения к метрикам:**

**1. Метрики фрустрации и сложности:**
    *   **Rollback Frequency (Частота отката):** Как часто игроки сталкиваются с откатом? Среднее количество откатов на одного участника.
    *   **Average Rollback Depth (Средняя глубина отката):** На сколько уровней в среднем откатывает игрока.
    *   **Levels with Highest Rollback Rate (Уровни с наибольшей частотой откатов):** Выявление особенно сложных или несбалансированных уровней, вызывающих частые неудачи и откаты.
    *   **Event Abandonment Rate (Коэффициент отказа от события):** `(Количество игроков, начавших событие, но прекративших участие до его завершения / Количество игроков, начавших событие) * 100%`. Эта метрика становится критически важной, так как высокая фрустрация может привести к тому, что игроки просто бросят событие. Нужно отслеживать, на каком этапе происходит основной отвал.
    *   **Attempts per Level (Количество попыток на уровень):** Особенно важно для уровней, предшествующих потенциальному откату. Увеличение этого показателя может сигнализировать о проблемах.

**2. Метрики прогрессии (с учетом новой механики):**
    *   **"True" Progress Rate (Реальная скорость прогрессии):** Сколько уровней игрок проходит "чистыми" за единицу времени или сессию, учитывая откаты.
    *   **Event Completion Rate:** Ожидаемо может снизиться. Важно сравнить с предыдущими событиями, чтобы понять масштаб влияния новой механики.

**3. Влияние на удержание и отток (усиленный фокус):**
    *   **Churn Rate among Event Participants (Уровень оттока среди участников события):** Необходимо пристально следить, не приводит ли повышенная сложность и фрустрация к уходу игроков из игры в целом. Сравнить с оттоком игроков, не участвовавших в событии.
    *   **Short-term Retention Post-Event (Краткосрочное удержание после события):** Как быстро игроки возвращаются в игру после такого (потенциально стрессового) события.

**4. Монетизация (потенциально новые аспекты):**
    *   **Sales of "Anti-Rollback" Items (Продажи предметов, предотвращающих откат):** Если в игру вводятся специальные платные предметы или бустеры, которые помогают избежать отката или смягчают его последствия, то их продажи станут важной метрикой.

**5. Качественные метрики (особое внимание):**
    *   **Sentiment Analysis on "Rollback" Mechanic (Анализ настроений по поводу механики отката):** Сбор отзывов именно об этой механике. Была ли она воспринята как интересный вызов или как неоправданно карающая.